In [1]:
pip install langchain openai python-dotenv requests duckduckgo-search langchain-openai langchainhub gradio_tools

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from langchain.tools import Tool, DuckDuckGoSearchResults
from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain.agents import AgentExecutor, create_react_agent
from langchain import hub
from langchain_core.prompts import PromptTemplate
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from gradio_tools import BarkTextToSpeechTool

In [3]:
OPENAI_API_KEY="your_api_key"

In [4]:
ddg_search = DuckDuckGoSearchResults()

In [5]:
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:90.0) Gecko/20100101 Firefox/90.0'
}

In [6]:
def parse_html(content) -> str:
    soup = BeautifulSoup(content, 'html.parser')
    text_content_with_links = soup.get_text()
    return text_content_with_links

In [7]:
def fetch_web_page(url: str) -> str:
    response = requests.get(url, headers=HEADERS)
    return parse_html(response.content)

In [8]:
web_fetch_tool = Tool.from_function(
    func=fetch_web_page,
    name="WebFetcher",
    description="Fetches the content of a web page"
)

In [9]:
prompt_template = "Summarize the following content: {content}"
llm = OpenAI(api_key= OPENAI_API_KEY)
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)

summarize_tool = Tool.from_function(
    func=llm_chain.run,
    name="Summarizer",
    description="Summarizes a web page"
)


In [10]:
template = '''Answer the following questions as best you can. Please keep the maximum context length to 4097 tokens.
You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer\n
Thought: you should always think about what to do\n
Action: the action to take, should be one of [{tool_names}]\n
Action Input: the input to the action , make it as short as possible.\n
Observation: the result of the action\n
... (this Thought/Action/Action Input/Observation can repeat N times)\n
Thought: I now know the final answer\n
Final Answer: the final answer to the original input question\n

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

In [11]:
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], template='Answer the following questions as best you can. Please keep the maximum context length to 4097 tokens.\nYou have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\n\nThought: you should always think about what to do\n\nAction: the action to take, should be one of [{tool_names}]\n\nAction Input: the input to the action , make it as short as possible.\n\nObservation: the result of the action\n\n... (this Thought/Action/Action Input/Observation can repeat N times)\n\nThought: I now know the final answer\n\nFinal Answer: the final answer to the original input question\n\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [12]:
tools = [ddg_search, web_fetch_tool, summarize_tool , BarkTextToSpeechTool().langchain]
agent = create_react_agent(llm, tools, prompt)

Loaded as API: https://suno-bark.hf.space ✔


In [14]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke(
    {
        "input": "Research how to use the requests library in Python. "
        "Use your tools to search and summarize content into a guide on how to use the requests library."
        "Then generate a speech in English for that summary."
    }
)



> Entering new AgentExecutor chain...
 I should start by using the Duck Duck Go Search tool to find relevant information about the requests library in Python.
Action: duckduckgo_results_json
Action Input: "How to use the requests library in Python"[snippet: Requests is an Apache2 Licensed HTTP library, that allows to send HTTP/1.1 requests using Python. To play with web, Python Requests is must. Whether it be hitting APIs, downloading entire facebook pages, and much more cool stuff, one will have to make a request to the URL., title: Python Requests Tutorial - GeeksforGeeks, link: https://www.geeksforgeeks.org/python-requests-tutorial/], [snippet: Follow these steps to use the Python requests module. Install the Python Requests Package. $ pip install requests. Import the Requests Module. import requests. Make a Request using the GET method. Use the GET method and store the response in a variable., title: How to Use Python Requests Library (Example and Video) - Guided ..., link: https

{'input': 'Research how to use the requests library in Python. Use your tools to search and summarize content into a guide on how to use the requests library.Then generate a speech in English for that summary.',
 'output': 'The final answer is a summarized guide on how to use the requests library in Python, generated by the Summarizer tool and converted into a speech in English using the BarkTextToSpeech tool.'}

In [18]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke(
    {
        "input": "Research how to make a youtube video."
        "Use your tools to search and summarize content into a guide on how to make a youtube video."
        "Then generate a speech in English for that summary."
    }
)



> Entering new AgentExecutor chain...
 I should use the tools available to me to search for information on how to make a YouTube video, summarize the content found, and generate a speech in English.
Action: duckduckgo_results_json
Action Input: How to make a YouTube video[snippet: Learn how to use free video editors to properly export content for a YouTube channel in the right format. Find out the steps, tools, and tips for creating captivating videos with Shotcut, OpenShot, Apple iMovie, HitFilm, DaVinci Resolve and more., title: How to make a YouTube video | TechRadar, link: https://www.techradar.com/pro/software-services/how-to-make-a-youtube-video], [snippet: Welcome to our comprehensive guide on creating the perfect YouTube video! Whether you're a seasoned content creator or just starting out, this video is packe..., title: HOW TO MAKE A PERFECT YOUTUBE VIDEO? - YouTube, link: https://www.youtube.com/watch?v=Eac-bnBr8Bk], [snippet: Tags are keywords that help YouTube understand 

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: ` I should listen to the generated speech and take notes.
Action: Final Answer`

In [21]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke(
    {
        "input": "Research how to write a resume."
        "Use your tools to search and summarize content into a guide on how to write a resume."
        "Then generate a speech in English for that summary."
    }
)



> Entering new AgentExecutor chain...
 To answer this question, I will need to use the duckduckgo_results_json to search for relevant information, the Summarizer to condense the information into a guide, and the BarkTextToSpeech to generate a speech in English.
Action: duckduckgo_results_json
Action Input: "how to write a resume"[snippet: Learn how to create a resume that communicates your skills and qualifications to employers. Follow steps and best practices for choosing the right format, including contact information, summary or objective, skills, professional history, education and optional sections., title: How To Make a Comprehensive Resume (With Examples), link: https://www.indeed.com/career-advice/resumes-cover-letters/how-to-make-a-resume-with-examples], [snippet: Learn how to write a resume in 10 steps with this comprehensive guide. Choose a resume format, list your contact information, summarize your value, showcase your skills, and more., title: How to Make a Great Resume

{'input': 'Research how to write a resume.Use your tools to search and summarize content into a guide on how to write a resume.Then generate a speech in English for that summary.',
 'output': 'To write a resume, follow these steps according to various guides: Choose the right resume format, include your contact information, write a summary or objective, showcase your skills, list your professional history, and include any relevant education and optional sections. These steps will help you create a comprehensive and impressive resume that effectively communicates your skills and qualifications to potential employers.'}

In [40]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke(
    {
        "input": "Research about the rivalry between Galatasaray and Fenerbahce."
        "Use your tools to search and summarize content about the rivalry between Galatasaray and Fenerbahce."
        "Then generate a speech in English for that summary."
    }
)



> Entering new AgentExecutor chain...
 To answer this question, I should use the tools that are available to me to gather information about the rivalry between Galatasaray and Fenerbahce and then use the summarizer tool to generate a concise summary. Finally, I can use the BarkTextToSpeech tool to convert the summary into a speech in English.

Action: duckduckgo_results_json
Action Input: Galatasaray and Fenerbahce rivalry[snippet: The Intercontinental Derby (Turkish: Kıtalararası Derbi) is any football match between rivals Fenerbahçe SK and Galatasaray SK. The fixture is widely regarded as the biggest football match in Turkey, and one of the biggest matches in international football because of the success both clubs have had in Turkish football, the intensity of the matches, and the immense rivalry between the two teams., title: The Intercontinental Derby (football) - Wikipedia, link: https://en.wikipedia.org/wiki/The_Intercontinental_Derby_(football)], [snippet: It's a rivalry that

{'input': 'Research about the rivalry between Galatasaray and Fenerbahce.Use your tools to search and summarize content about the rivalry between Galatasaray and Fenerbahce.Then generate a speech in English for that summary.',
 'output': 'The rivalry between Galatasaray and Fenerbahce is a long-standing and intense competition between two popular Turkish football clubs. These teams have a history of heated matches and fierce competition for titles and bragging rights. The rivalry is fueled by the fact that both clubs represent different regions of Istanbul and have large fan bases. Despite the intense rivalry, both teams have shown sportsmanship and respect towards each other, making it a unique and highly anticipated event in the Turkish sports world.'}